In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Build the tokenizer

In [8]:
import os
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers import Concatenate

In [9]:
os.chdir('/content/drive/MyDrive/ICON_Task/Tamil')
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

Files in '/content/drive/.shortcut-targets-by-id/1i5k6Jb01kvneiUkS2K-1eZBVNfG0RVR4/Tamil': ['train_ta_l1.csv', 'glove', 'fasttext', 'tamil_train_combined - tamil_train.csv', 'train_ta_l1_final.gsheet', 'train_ta_l1_final - train_ta_l1.csv', 'tamil_MACD_Combined.csv', 'train_ta_l1_modded.gsheet', 'train_ta_l1_modded.csv', 'Fb_FastText', 'CNN_BiLSTM_ICON_Task-2_Tamil_Binary_Classification.h5', 'New_CNN+BiLSTM_Attempt-2_Tamil.ipynb', 'Tamil_Task2_output.csv', 'CNN_BiLSTM_ICON_Task-1_Tamil_Binary_Classification.h5', 'Tamil_Task1_output.csv', 'New_CNN+BiLSTM_Attempt-1_Tamil.ipynb']


In [12]:
df = pd.read_csv('train_ta_l1_final - train_ta_l1.csv')
df = df.drop(['key', 'ta_a1', 'ta_a2', 'ta_a3', 'ta_a4', 'ta_a5', 'ta_a6'], axis=1)
df['Label'] = df['Label'].astype(int)
df = df[['Label', 'text']]
df

,Label,text
0,0,*1. முரசொலி அலுவலகம் அமைந்துள்ள இடம் பஞ்சமி நி...
1,0,சோத்துக்கு பிச்சை எடுக்கிற கடங்கார நாய்களுக்கு...
2,0,தத்தபுத்த தத்தபுத்த ன்னு எதாவது புரியுதா
3,1,பச்சை மொளகா காரம் vicky அம்மா புண்டை நாறும் 😆
4,1,என்ன உடம்பு டா சாமி- சும்மா வளுவளுனு.. முலை ஒவ...
...,...,...
6774,1,😭😭😭 ஒம்மாள படிக்கல் புண்ட 😭😭😭
6775,0,🙄🙄🙄🙄 என்ன எழவுயா இது... இதெல்லாம் ஒரு பெருமை...
6776,0,🚨எக்ஸ் பிரஸ் பேர்ல் கப்பல் தீ விபத்துக்கு உள்ள...
6777,0,🤣 🤣 சல்லி ஜாதி வெறி முட்டா புண்ட உங்க பொண்ணுங்...


In [13]:
df.rename(columns = {'Label':'label'}, inplace = True)
df

,label,text
0,0,*1. முரசொலி அலுவலகம் அமைந்துள்ள இடம் பஞ்சமி நி...
1,0,சோத்துக்கு பிச்சை எடுக்கிற கடங்கார நாய்களுக்கு...
2,0,தத்தபுத்த தத்தபுத்த ன்னு எதாவது புரியுதா
3,1,பச்சை மொளகா காரம் vicky அம்மா புண்டை நாறும் 😆
4,1,என்ன உடம்பு டா சாமி- சும்மா வளுவளுனு.. முலை ஒவ...
...,...,...
6774,1,😭😭😭 ஒம்மாள படிக்கல் புண்ட 😭😭😭
6775,0,🙄🙄🙄🙄 என்ன எழவுயா இது... இதெல்லாம் ஒரு பெருமை...
6776,0,🚨எக்ஸ் பிரஸ் பேர்ல் கப்பல் தீ விபத்துக்கு உள்ள...
6777,0,🤣 🤣 சல்லி ஜாதி வெறி முட்டா புண்ட உங்க பொண்ணுங்...


In [14]:
## Word Pre-Processing ##
import nltk
import string
import re

## Function to normalize text for pre-processing ##
def normalize_text(text):
  emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F700-\U0001F77F"  # alchemical symbols
                              u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                              u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                              u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                              u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                              u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                              u"\U00002702-\U000027B0"  # Dingbats
                              u"\U000024C2-\U0001F251"
                              "]+", flags=re.UNICODE)
  text = text.lower()
  text = re.sub('\[.*?\]', ' ', text)
  text = re.sub('https?://\S+|www\.\S+', ' ', text)
  text = re.sub('<.*?>+', ' ', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
  text = re.sub('\n', ' ', text)
  text = re.sub('\w*\d\w*', ' ', text)
  text = re.sub('<handle replaced>', '', text)
  text = emoji_pattern.sub(r'', text)
  return text
## Apply the written function ##
df['text'] = df['text'].apply(lambda x: normalize_text(x))
df

,label,text
0,0,முரசொலி அலுவலகம் அமைந்துள்ள இடம் பஞ்சமி நி...
1,0,சோத்துக்கு பிச்சை எடுக்கிற கடங்கார நாய்களுக்கு...
2,0,தத்தபுத்த தத்தபுத்த ன்னு எதாவது புரியுதா
3,1,பச்சை மொளகா காரம் vicky அம்மா புண்டை நாறும்
4,1,என்ன உடம்பு டா சாமி சும்மா வளுவளுனு முலை ஒவ...
...,...,...
6774,1,ஒம்மாள படிக்கல் புண்ட
6775,0,என்ன எழவுயா இது இதெல்லாம் ஒரு பெருமைனு ந...
6776,0,எக்ஸ் பிரஸ் பேர்ல் கப்பல் தீ விபத்துக்கு உள்ளா...
6777,0,சல்லி ஜாதி வெறி முட்டா புண்ட உங்க பொண்ணுங்க ...


In [15]:
X = list(df['text'])
#y = np.asarray(d1[d1.columns[1:]])
y_data = df['label']
y = y_data.values

In [16]:
max_seq_length = 100
max_features = 33064
embedding_dim = 300

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X))
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, padding='post', maxlen=max_seq_length)

# Load the Trained Model

In [22]:
os.chdir('/content/drive/MyDrive/ICON_Task/Tamil')
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

Files in '/content/drive/.shortcut-targets-by-id/1i5k6Jb01kvneiUkS2K-1eZBVNfG0RVR4/Tamil': ['train_ta_l1.csv', 'glove', 'fasttext', 'tamil_train_combined - tamil_train.csv', 'train_ta_l1_final.gsheet', 'train_ta_l1_final - train_ta_l1.csv', 'tamil_MACD_Combined.csv', 'train_ta_l1_modded.gsheet', 'train_ta_l1_modded.csv', 'Fb_FastText', 'CNN_BiLSTM_ICON_Task-2_Tamil_Binary_Classification.h5', 'New_CNN+BiLSTM_Attempt-2_Tamil.ipynb', 'Tamil_Task2_output.csv', 'CNN_BiLSTM_ICON_Task-1_Tamil_Binary_Classification.h5', 'Tamil_Task1_output.csv', 'New_CNN+BiLSTM_Attempt-1_Tamil.ipynb', 'CNN_BiLSTM_ICON_Task-3_Tamil_Binary_Classification.h5']


In [24]:
model = keras.models.load_model('/content/drive/MyDrive/ICON_Task/Tamil/CNN_BiLSTM_ICON_Task-3_Tamil_Binary_Classification.h5')

# Testing on Test Data

In [29]:
import pandas as pd
vv= pd.read_csv('/content/test_label1.csv')
vv

,id,text,language,key
0,1,"""who's this tweet about"" bitch it's about you ...",english,question_1
1,2,#AlkaLamba should’ve used her slippers to slap...,english,question_1
2,3,#CoronaVillains #COVIDIDIOT #COVID2019 is a...,english,question_1
3,4,"(Acts 29:19994) LIKE the female ""governors"" of...",english,question_1
4,5,***Let us celebrate “secular” as a beautiful w...,english,question_1
...,...,...,...,...
3753,3754,😂😂😂 ஊம்பு,tamil,question_1
3754,3755,"😄 தமிழ் தெரிஞ்சவன்""தான் உங்கொம்மால குண்டி அடிக...",tamil,question_1
3755,3756,😅😅😂😂 நீ தான் பங் அவனோலுக்கு கரெக்டா ஆன ஆளு.. 😎...,tamil,question_1
3756,3757,😺✏ — ஆமாம் வேச ஆமாம் வேச அய்ம் ச்லீபின் அய்ம் ...,tamil,question_1


In [30]:
vv = vv[vv['language'] == 'tamil']
vv

,id,text,language,key
2623,2624,வைரமுத்து ஒரு காம மிருகம் என்பது சினிமா துற...,tamil,question_1
2624,2625,#4YrsOfValiantVIVEGAM #Valimai #AjithKumar ...,tamil,question_1
2625,2626,#AmbedkarBlueShirtRally இந்த போராட்டத்துக்கு ...,tamil,question_1
2626,2627,#BREAKING | திருச்சி மாவட்டம் மணப்பாறையை அடுத...,tamil,question_1
2627,2628,#Bachelor 😤😤😤😤😤படமாடா இது கோத்தா <handle repla...,tamil,question_1
...,...,...,...,...
3753,3754,😂😂😂 ஊம்பு,tamil,question_1
3754,3755,"😄 தமிழ் தெரிஞ்சவன்""தான் உங்கொம்மால குண்டி அடிக...",tamil,question_1
3755,3756,😅😅😂😂 நீ தான் பங் அவனோலுக்கு கரெக்டா ஆன ஆளு.. 😎...,tamil,question_1
3756,3757,😺✏ — ஆமாம் வேச ஆமாம் வேச அய்ம் ச்லீபின் அய்ம் ...,tamil,question_1


In [31]:
vv1 = vv
vv1

,id,text,language,key
2623,2624,வைரமுத்து ஒரு காம மிருகம் என்பது சினிமா துற...,tamil,question_1
2624,2625,#4YrsOfValiantVIVEGAM #Valimai #AjithKumar ...,tamil,question_1
2625,2626,#AmbedkarBlueShirtRally இந்த போராட்டத்துக்கு ...,tamil,question_1
2626,2627,#BREAKING | திருச்சி மாவட்டம் மணப்பாறையை அடுத...,tamil,question_1
2627,2628,#Bachelor 😤😤😤😤😤படமாடா இது கோத்தா <handle repla...,tamil,question_1
...,...,...,...,...
3753,3754,😂😂😂 ஊம்பு,tamil,question_1
3754,3755,"😄 தமிழ் தெரிஞ்சவன்""தான் உங்கொம்மால குண்டி அடிக...",tamil,question_1
3755,3756,😅😅😂😂 நீ தான் பங் அவனோலுக்கு கரெக்டா ஆன ஆளு.. 😎...,tamil,question_1
3756,3757,😺✏ — ஆமாம் வேச ஆமாம் வேச அய்ம் ச்லீபின் அய்ம் ...,tamil,question_1


In [32]:
## Word Pre-Processing ##
import nltk
import string
import re

## Function to normalize text for pre-processing ##
def normalize_text(text):
  emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F700-\U0001F77F"  # alchemical symbols
                              u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                              u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                              u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                              u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                              u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                              u"\U00002702-\U000027B0"  # Dingbats
                              u"\U000024C2-\U0001F251"
                              "]+", flags=re.UNICODE)
  text = text.lower()
  text = re.sub('\[.*?\]', ' ', text)
  text = re.sub('https?://\S+|www\.\S+', ' ', text)
  text = re.sub('<.*?>+', ' ', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
  text = re.sub('\n', ' ', text)
  text = re.sub('\w*\d\w*', ' ', text)
  text = re.sub('<handle replaced>', '', text)
  text = emoji_pattern.sub(r'', text)
  return text
## Apply the written function ##
vv1['text'] = vv1['text'].apply(lambda x: normalize_text(x))
vv1

<ipython-input-32-01992e7e1f18>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vv1['text'] = vv1['text'].apply(lambda x: normalize_text(x))


,id,text,language,key
2623,2624,வைரமுத்து ஒரு காம மிருகம் என்பது சினிமா துற...,tamil,question_1
2624,2625,valimai ajithkumar ஏண்டா குள்ள புண்டைக...,tamil,question_1
2625,2626,ambedkarblueshirtrally இந்த போராட்டத்துக்கு ...,tamil,question_1
2626,2627,breaking திருச்சி மாவட்டம் மணப்பாறையை அடுத...,tamil,question_1
2627,2628,bachelor படமாடா இது கோத்தா,tamil,question_1
...,...,...,...,...
3753,3754,ஊம்பு,tamil,question_1
3754,3755,தமிழ் தெரிஞ்சவன் தான் உங்கொம்மால குண்டி அடிக்...,tamil,question_1
3755,3756,நீ தான் பங் அவனோலுக்கு கரெக்டா ஆன ஆளு லத்த...,tamil,question_1
3756,3757,— ஆமாம் வேச ஆமாம் வேச அய்ம் ச்லீபின் அய்ம் ச்...,tamil,question_1


In [33]:
######## Textual Features for Embedding ###################
sequence = tokenizer.texts_to_sequences(vv1.text)
sequence = pad_sequences(sequence, padding='post', maxlen=max_seq_length)

In [34]:
sequence = tokenizer.texts_to_sequences(vv1['text']) # same tokenizer which is used on train data.
sequence = pad_sequences(sequence,padding='post', maxlen = max_seq_length)

In [35]:
import numpy as np

predictions = model.predict(sequence)
classes=np.argmax(predictions,axis=1)

36/36 [==============================] - 5s 112ms/step


In [36]:
predictions

array([[0.3202512 , 0.6797487 ],
       [0.39385137, 0.6061487 ],
       [0.85223526, 0.14776476],
       ...,
       [0.30137563, 0.6986244 ],
       [0.76338834, 0.23661163],
       [0.68694234, 0.3130577 ]], dtype=float32)

In [37]:
pred = np.argmax(predictions, axis=1)
pred

array([1, 1, 0, ..., 1, 0, 0])

In [38]:
ids = np.arange(2624, 3759)
# Create a DataFrame with 'id' and 'value' columns
result_df = pd.DataFrame({'id': ids, 'value': pred})
# result_df = pd.DataFrame({'value': predictions})
result_df

,id,value
0,2624,1
1,2625,1
2,2626,0
3,2627,0
4,2628,1
...,...,...
1130,3754,1
1131,3755,1
1132,3756,1
1133,3757,0


In [39]:
# Save the DataFrame to a CSV file
result_df.to_csv('/content/Tamil_Task3-l1_output.csv', index=False)

In [40]:
vv2= pd.read_csv('/content/test_label3.csv')
vv2

,id,text,language,key
0,1,"""who's this tweet about"" bitch it's about you ...",english,question_3
1,2,#AlkaLamba should’ve used her slippers to slap...,english,question_3
2,3,#CoronaVillains #COVIDIDIOT #COVID2019 is a...,english,question_3
3,4,"(Acts 29:19994) LIKE the female ""governors"" of...",english,question_3
4,5,***Let us celebrate “secular” as a beautiful w...,english,question_3
...,...,...,...,...
3753,3754,😂😂😂 ஊம்பு,tamil,question_3
3754,3755,"😄 தமிழ் தெரிஞ்சவன்""தான் உங்கொம்மால குண்டி அடிக...",tamil,question_3
3755,3756,😅😅😂😂 நீ தான் பங் அவனோலுக்கு கரெக்டா ஆன ஆளு.. 😎...,tamil,question_3
3756,3757,😺✏ — ஆமாம் வேச ஆமாம் வேச அய்ம் ச்லீபின் அய்ம் ...,tamil,question_3


In [41]:
vv2 = vv2[vv2['language'] == 'tamil']
vv2

,id,text,language,key
2623,2624,வைரமுத்து ஒரு காம மிருகம் என்பது சினிமா துற...,tamil,question_3
2624,2625,#4YrsOfValiantVIVEGAM #Valimai #AjithKumar ...,tamil,question_3
2625,2626,#AmbedkarBlueShirtRally இந்த போராட்டத்துக்கு ...,tamil,question_3
2626,2627,#BREAKING | திருச்சி மாவட்டம் மணப்பாறையை அடுத...,tamil,question_3
2627,2628,#Bachelor 😤😤😤😤😤படமாடா இது கோத்தா <handle repla...,tamil,question_3
...,...,...,...,...
3753,3754,😂😂😂 ஊம்பு,tamil,question_3
3754,3755,"😄 தமிழ் தெரிஞ்சவன்""தான் உங்கொம்மால குண்டி அடிக...",tamil,question_3
3755,3756,😅😅😂😂 நீ தான் பங் அவனோலுக்கு கரெக்டா ஆன ஆளு.. 😎...,tamil,question_3
3756,3757,😺✏ — ஆமாம் வேச ஆமாம் வேச அய்ம் ச்லீபின் அய்ம் ...,tamil,question_3


In [42]:
## Word Pre-Processing ##
import nltk
import string
import re

## Function to normalize text for pre-processing ##
def normalize_text(text):
  emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F700-\U0001F77F"  # alchemical symbols
                              u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                              u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                              u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                              u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                              u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                              u"\U00002702-\U000027B0"  # Dingbats
                              u"\U000024C2-\U0001F251"
                              "]+", flags=re.UNICODE)
  text = text.lower()
  text = re.sub('\[.*?\]', ' ', text)
  text = re.sub('https?://\S+|www\.\S+', ' ', text)
  text = re.sub('<.*?>+', ' ', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
  text = re.sub('\n', ' ', text)
  text = re.sub('\w*\d\w*', ' ', text)
  text = re.sub('<handle replaced>', '', text)
  text = emoji_pattern.sub(r'', text)
  return text
## Apply the written function ##
vv2['text'] = vv2['text'].apply(lambda x: normalize_text(x))
vv2

<ipython-input-42-9ebfd64d3bb1>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vv2['text'] = vv2['text'].apply(lambda x: normalize_text(x))


,id,text,language,key
2623,2624,வைரமுத்து ஒரு காம மிருகம் என்பது சினிமா துற...,tamil,question_3
2624,2625,valimai ajithkumar ஏண்டா குள்ள புண்டைக...,tamil,question_3
2625,2626,ambedkarblueshirtrally இந்த போராட்டத்துக்கு ...,tamil,question_3
2626,2627,breaking திருச்சி மாவட்டம் மணப்பாறையை அடுத...,tamil,question_3
2627,2628,bachelor படமாடா இது கோத்தா,tamil,question_3
...,...,...,...,...
3753,3754,ஊம்பு,tamil,question_3
3754,3755,தமிழ் தெரிஞ்சவன் தான் உங்கொம்மால குண்டி அடிக்...,tamil,question_3
3755,3756,நீ தான் பங் அவனோலுக்கு கரெக்டா ஆன ஆளு லத்த...,tamil,question_3
3756,3757,— ஆமாம் வேச ஆமாம் வேச அய்ம் ச்லீபின் அய்ம் ச்...,tamil,question_3


In [43]:
######## Textual Features for Embedding ###################
sequence = tokenizer.texts_to_sequences(vv2.text)
sequence = pad_sequences(sequence, padding='post', maxlen=max_seq_length)

In [44]:
sequence = tokenizer.texts_to_sequences(vv2['text']) # same tokenizer which is used on train data.
sequence = pad_sequences(sequence,padding='post', maxlen = max_seq_length)

In [45]:
import numpy as np

predictions = model.predict(sequence)
classes=np.argmax(predictions,axis=1)

36/36 [==============================] - 3s 81ms/step


In [46]:
predictions

array([[0.3202512 , 0.6797487 ],
       [0.39385137, 0.6061487 ],
       [0.85223526, 0.14776476],
       ...,
       [0.30137563, 0.6986244 ],
       [0.76338834, 0.23661163],
       [0.68694234, 0.3130577 ]], dtype=float32)

In [47]:
pred = np.argmax(predictions, axis=1)
pred

array([1, 1, 0, ..., 1, 0, 0])

In [48]:
ids = np.arange(2624, 3759)
# Create a DataFrame with 'id' and 'value' columns
result_df = pd.DataFrame({'id': ids, 'value': pred})
# result_df = pd.DataFrame({'value': predictions})
result_df

,id,value
0,2624,1
1,2625,1
2,2626,0
3,2627,0
4,2628,1
...,...,...
1130,3754,1
1131,3755,1
1132,3756,1
1133,3757,0


In [49]:
# Save the DataFrame to a CSV file
result_df.to_csv('/content/Tamil_Task3-l3_output.csv', index=False)